In [1]:
#Results were produced in stints. This is the number of the last stint.
run = 3

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy import stats
import joblib

#folder for saving results
filepath = ".../Resultate_final/PTF/saved_sim/"

In [3]:
#Market and option parameters as in section 4.2 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
s_0 = 100
r = 0.01
corr= 0.3
tau = 1/52
T = 1/3
K = 100

mu = np.empty(20)
sigma = np.empty(20)
for i in range(0,10):
    mu[i] = mu[i+10] = (3+(i+1)/2)/100
    sigma[i] = sigma[i+10] = (15+(i+1))/100

cov_mat = np.empty((20,20))
for i in range(0,20):
    for j in range(0,20):
        if i != j:
            cov_mat[i,j] = corr
        else:
            cov_mat[i,j] = 1

C = np.linalg.cholesky(cov_mat)

#Confidence levels and parameters for Value-at-Risk, Expected Shortfall and GlueVaR
alpha_VaR = 0.995
alpha_ES = 0.99
alpha_Glue = 0.95
beta_Glue = 0.995
omega_Glue = np.array([1/3,1/3])

In [4]:
#Sizes for training set, validation set, test set, and set size for Monte Carlo estimation of the risk measures
M_1 = 1500000
M_2 = 500000
M_3 = 500000
#ignore N or N_2 in the following. Was kept just in case, but not used.
N_2 = 1
M_MC = 500000

In [5]:
#Function for calculating simulated values of S_tau and simulated payoffs P_T from simulations of multivariate standard normal random variables
def data_gen(M,N,Z,V):
    #correlating the independent components
    Z = np.transpose(np.matmul(C,np.transpose(Z)))
    V = np.transpose(np.matmul(C,np.transpose(V)))

    #simulate S_tau under P
    S_tau_pre = np.empty((M, 20))
    for j in range(0,20):
        S_tau_pre[:,j] = s_0 * np.exp( (mu[j]-0.5*sigma[j]**2)*tau + np.sqrt(tau)*sigma[j]*Z[:,j] )
    S_tau = np.tile(S_tau_pre, (N,1))

    #simulate S_T  given S_tau under Q
    S_T = np.empty((N*M,20))
    for j in range(0,20):
        S_T[:,j] = S_tau[:,j] * np.exp( (r-0.5*sigma[j]**2)*(T-tau) + np.sqrt(T-tau)*sigma[j]*V[:,j] )

    #compute discounted option payoffs
    P_T_pre =np.empty((len(S_T), 20))
    for j in range(0,10):
        P_T_pre[:,j] = np.exp(-r*(T-tau)) * np.maximum(S_T[:,j]-K,0)
    for j in range(10,20):
        P_T_pre[:,j] = np.exp(-r*(T-tau)) * np.maximum(K-S_T[:,j],0)
    P_T = np.sum(P_T_pre, axis=1)
    return S_tau,P_T

#Function for the computation of GlueVaR
def GlueVaR(omega, L, alpha, beta):
    j_beta = int(len(L)*(1-beta))-1
    j_alpha = int(len(L)*(1-alpha))-1

    ES_beta = 1/(1-beta) * np.sum(L[0:j_beta-1])/len(L) + ( 1 - (j_beta-1)/((1-beta)*len(L)) )*L[j_beta]
    ES_alpha = 1/(1-alpha) * np.sum(L[0:j_alpha-1])/len(L) + ( 1 - (j_alpha-1)/((1-alpha)*len(L)) )*L[j_alpha]
    VaR_alpha = L[j_alpha]

    return omega[0]*ES_beta + omega[1]*ES_alpha + (1-omega[0]-omega[1])*VaR_alpha

In [6]:
for j in range(100):
    #simulation of multivariate standard normal random variables
    Z_train = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=int(M_1/N_2))
    V_train = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_1)
    Z_val = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_2)
    V_val = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_2)
    Z_test = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_3)
    V_test = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_3)
    Z_MC = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_MC)
    V_MC = np.random.multivariate_normal(mean=np.full(20,0), cov=np.identity(20), size=M_MC)

    #Calculate the risk factor S_tau and the corresponding simulated payoffs P_T
    S_tau_train, P_T_train = data_gen(M=int(M_1/N_2), N=N_2, Z=Z_train, V=V_train)
    S_tau_val, P_T_val = data_gen(M=M_2, N=1, Z=Z_val, V=V_val)
    S_tau_test, P_T_test = data_gen(M=M_3, N=1, Z=Z_test, V=V_test)
    S_tau_MC, P_T_MC = data_gen(M=M_MC, N=1, Z=Z_MC, V=V_MC)


    #define and compile neural network model, setup as in section 4.2 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
    bi = np.log( np.sum(P_T_train)/len(P_T_train) )
    model = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(20,)),
        tf.keras.layers.Dense(20, activation='tanh'),
        tf.keras.layers.Dense(20, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi))])
    #compilation
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #training
    hist = model.fit(x=S_tau_train, y=P_T_train, epochs=100, batch_size=10000, validation_data=(S_tau_val,P_T_val), verbose=0)

    #definition of random forest regressor
    rfr = RandomForestRegressor(n_estimators=400, criterion='squared_error', max_features=6, min_samples_leaf=50, bootstrap=True, verbose=0, warm_start=True, n_jobs=-1)
    #training
    rfr.fit(X=S_tau_train, y=P_T_train)

    #Calculating the parts of the test set that fall into B_1 and B_2
    s_20_1 = s_0 * np.exp((mu[0:3]-0.5*sigma[0:3]**2)*tau + sigma[0:3]*np.sqrt(tau)*stats.norm.ppf(0.2, loc=0, scale=1))
    s_80_1 = s_0 * np.exp((mu[9:12]-0.5*sigma[9:12]**2)*tau + sigma[9:12]*np.sqrt(tau)*stats.norm.ppf(0.8, loc=0, scale=1))
    s_20_2 = s_0 * np.exp((mu[9:12]-0.5*sigma[9:12]**2)*tau + sigma[9:12]*np.sqrt(tau)*stats.norm.ppf(0.2, loc=0, scale=1))
    s_80_2 = s_0 * np.exp((mu[0:3]-0.5*sigma[0:3]**2)*tau + sigma[0:3]*np.sqrt(tau)*stats.norm.ppf(0.8, loc=0, scale=1))
    B_1 = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (np.apply_along_axis(np.all, axis=1, arr=S_tau_test[:,0:3] > s_20_1), np.apply_along_axis(np.all, axis=1, arr=S_tau_test[:,9:12] < s_80_1)) ) )
    B_2 = np.apply_along_axis(np.all, axis=1, arr=np.column_stack( (np.apply_along_axis(np.all, axis=1, arr=S_tau_test[:,0:3] < s_80_2), np.apply_along_axis(np.all, axis=1, arr=S_tau_test[:,9:12] > s_20_2)) ) )

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 for the neural network
    P_T_pred_NN = model.predict(S_tau_test)[:,0]
    mse_train_NN = hist.history['mse'][-1]
    mse_val_NN = hist.history['val_mse'][-1]
    print('Val MSE NN:',mse_val_NN)
    mc_tmp = P_T_pred_NN - P_T_test
    metric_a_NN = np.sum(mc_tmp)/len(P_T_test)
    metric_b_NN = np.sum((mc_tmp)*P_T_pred_NN)/len(P_T_test)
    metric_c_B_1_NN = np.sum(mc_tmp[B_1])/len(P_T_test)
    metric_c_B_2_NN = np.sum(mc_tmp[B_2])/len(P_T_test)

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 and training/validation MSE for the random forest
    mse_train_RF = mean_squared_error(y_pred=rfr.predict(S_tau_train),y_true=P_T_train)
    P_T_pred_RF = rfr.predict(S_tau_test)
    mse_val_RF = mean_squared_error(y_pred=P_T_pred_RF,y_true=P_T_test)
    print('Val MSE_RF:',mse_val_RF)
    mc_tmp = P_T_pred_RF - P_T_test
    metric_a_RF = np.sum(mc_tmp)/len(P_T_test)
    metric_b_RF = np.sum((mc_tmp)*P_T_pred_RF)/len(P_T_test)
    metric_c_B_1_RF = np.sum(mc_tmp[B_1])/len(P_T_test)
    metric_c_B_2_RF = np.sum(mc_tmp[B_2])/len(P_T_test)


    #computation of risk measure estimators
    #Prediction of total of all option prices
    P_T_pred_MC_NN = model.predict(S_tau_MC)[:,0]
    P_T_pred_MC_RF = rfr.predict(S_tau_MC)

    L_NN = np.sort(P_T_pred_MC_NN)[::-1]
    L_RF = np.sort(P_T_pred_MC_RF)[::-1]

    #Value-at-Risk
    j_VaR = int(M_MC*(1-alpha_VaR))-1
    VaR_hat_NN = L_NN[j_VaR]
    VaR_hat_RF = L_RF[j_VaR]
    
    #Expected Shortfall
    j_ES = int(M_MC*(1-alpha_ES))-1
    ES_hat_NN = 1/(1-alpha_ES) * np.sum(L_NN[0:j_ES-1])/M_MC + ( 1 - (j_ES-1)/((1-alpha_ES)*M_MC) )*L_NN[j_ES]
    ES_hat_RF = 1/(1-alpha_ES) * np.sum(L_RF[0:j_ES-1])/M_MC + ( 1 - (j_ES-1)/((1-alpha_ES)*M_MC) )*L_RF[j_ES]
    
    #GlueVaR
    GlueVaR_hat_NN = GlueVaR(omega=omega_Glue, L=L_NN, alpha=alpha_Glue, beta=beta_Glue)
    GlueVaR_hat_RF = GlueVaR(omega=omega_Glue, L=L_RF, alpha=alpha_Glue, beta=beta_Glue)
    print('Estimates NN:',VaR_hat_NN,ES_hat_NN,GlueVaR_hat_NN)
    print('Estimates RF:',VaR_hat_RF,ES_hat_RF,GlueVaR_hat_RF)

    #save results for further evaluation
    output = np.array([[mse_train_NN,mse_val_NN,metric_a_NN,metric_b_NN,metric_c_B_1_NN,metric_c_B_2_NN,VaR_hat_NN,ES_hat_NN,GlueVaR_hat_NN],
                      [mse_train_RF,mse_val_RF,metric_a_RF,metric_b_RF,metric_c_B_1_RF,metric_c_B_2_RF,VaR_hat_RF,ES_hat_RF,GlueVaR_hat_RF]])

    joblib.dump(output,filepath+'output_'+str(run)+'_'+str(j)+'.joblib')
    #prints just for checking while the notebook is running
    print(j)

2021-11-02 20:22:27.756455: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bwhpc/common/devel/cuda/11.4/lib64:/opt/bwhpc/common/compiler/gnu/10.2.0/lib:/opt/bwhpc/common/compiler/gnu/10.2.0/lib64:/opt/bwhpc/common/mpi/openmpi/4.1.1-gnu-10.2/lib:/opt/bwhpc/common/mpi/openmpi/4.1.1-gnu-10.2/lib64
2021-11-02 20:22:27.784077: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-02 20:22:27.784204: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (uc2n466.localdomain): /proc/driver/nvidia/version does not exist
2021-11-02 20:22:27.822815: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructio

Val MSE NN: 1157.6416015625
Val MSE_RF: 1163.8560339183705
Estimates NN: 109.00272 109.54302693176271 106.74819060099284
Estimates RF: 106.49996220376262 107.07083997866667 104.68165893966346
0
Val MSE NN: 1158.2738037109375
Val MSE_RF: 1162.3437348620853
Estimates NN: 110.53606 111.20630493774416 108.10699053487141
Estimates RF: 106.83561113430187 107.43731407597014 104.93267772368682
1
Val MSE NN: 1158.969482421875
Val MSE_RF: 1159.9496009184613
Estimates NN: 109.36378 109.86603711242677 106.99941491292319
Estimates RF: 106.96697098290224 107.609273616062 105.03237104905057
2
Val MSE NN: 1158.126708984375
Val MSE_RF: 1158.5275813405767
Estimates NN: 110.43596 111.03008006896974 107.95805806091309
Estimates RF: 106.75125545043244 107.26398291129676 104.80639099728197
3
Val MSE NN: 1160.1044921875
Val MSE_RF: 1158.6566252854618
Estimates NN: 110.767265 111.41791625366211 108.10700551635742
Estimates RF: 106.81646370948465 107.44119615623408 104.89860693839353
4
Val MSE NN: 1154.8298339

KeyboardInterrupt: 